In [1]:
from flask import Flask, request, render_template, make_response , jsonify
import pymongo
import json
from json import dumps
import simplejson as json
from bson import json_util
from bson.objectid import ObjectId
from bson import ObjectId
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Random import get_random_bytes
import gridfs
import datetime
import werkzeug
from tkinter import *
import tkinter as tk
from tika import parser
import operator
from urllib.request import urlopen
from pprint import pprint
from bs4 import BeautifulSoup
import json
from flask_cors import CORS, cross_origin
import os


# Initialization

In [2]:
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

# RSA 4096 bits encryption with e=65537

UPLOAD_FOLDER = r'F:\Book-guide web form\Upload'
app.config['UPLOAD_EXTENSIONS'] = ['.jpg', '.png', '.pdf', '.docx']
app.config['MAX_CONTENT_LENGTH'] = 25*1000000 # 25 MB Limit
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
mng_client = pymongo.MongoClient('localhost', 27017)

pub_key_file = open('public_key.pem','r')
pvt_key_file = open('private_key.pem','r')
pubkey = RSA.importKey(pub_key_file.read())
pvtkey = RSA.importKey(pvt_key_file.read())
encryptor = PKCS1_OAEP.new(pubkey)
decryptor = PKCS1_OAEP.new(pvtkey)

def decryption(encrypted_message):
    return decryptor.decrypt(encrypted_message).decode("utf-8")

def encryption(plain_message):
    return encryptor.encrypt(bytes(plain_message,'utf-8'))

# Test Area

In [3]:
@app.route('/student/pdf', methods = ['POST'])
def file_upload():
    if request.method== 'POST':
        files_get=request.files['pdf']
        filename = werkzeug.utils.secure_filename(files_get.filename)
        files_get.save(filename)
        print(files_get)
        response_string = {"status": "success"}
    else:
        response_string = {"status": "failed"}
    return json.dumps(response_string, indent=4)

# Welcome API

In [4]:
@app.route('/')
def welcome():
    return "<b>Welcome to Jakstech API Service!</b>"

# Admin - registration

In [ ]:
@app.route('/admin/registration',methods=["POST"])
def admin_registration():
#   declaration space
    db = myclient["accounts"]
    collection = db["admin"]
    data = {}
    response_string = {}
    
    if request.method == 'POST':
        data = {
            "name": request.form['name'],
            "dob": request.form['dob'],
            "sex": request.form['sex'],
            "admin_id": request.form['admin_id'], 
            "schoolname": request.form['schoolname'],
            "aadhar": request.form['aadhar'], 
            "mobile": request.form['mobile'],
            "password": request.form['password'],  
#             "encrypted_pwd": encryptor.encrypt(bytes(request.form['password'],'utf-8')),    
            "createdAt": str(datetime.datetime.now()).split(" ")[0],                        #YYYY-MM-DD
            "loggedIn": False,
            "address": request.form['address'],
            "pincode": request.form['pincode']
        }
        
        db_check_user = collection.find_one({"admin_id":data["admin_id"]}) #check if user account exists
        if not(db_check_user):
            db_response = collection.insert_one(data)
            
            if db_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
        
        else:
            response_string = {"status": "failed","response": "User account already exists"}
        
    return json.dumps(response_string)

# Admin - login

In [ ]:
@app.route('/admin/login',methods=["POST"])
def admin_login():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        adminId = request.form['admin_id']
        passw = request.form['password']
#         passw = decryption(request.form['password'])
        db_response = collection.find_one({"admin_id":adminId})
        if db_response != None:
            if db_response['loggedIn'] != True:
                if passw == db_response['password']:
                    update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": False},{"$set":{"loggedIn":True}})
                    if update_result.acknowledged == True and update_result.modified_count == 1:
                        response_string = {"status": "success","response":"ok"}
                    else:
                        response_string = {"status":"failed","response":"database error"}
                    return json.dumps(response_string, indent=4)
                else:
                    return json.dumps({"status": "failed","response":"wrong password"})
            else:
                return json.dumps({"status": "failed","response": "You are already logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

# Student Registration

In [5]:
@app.route('/admin-portal/student-registration',methods=["POST"])
def student_registration():
#   declaration space
    db = myclient["accounts"]
    collection = db["student"]
    collection_student = db["students_profile"]
    data = {}
    dataAccount = {}
    response_string = ""

        
    if request.method == 'POST':
        dataAccount = {
                "name": request.form['name'],
                "regno": request.form['regno'],
                "password": request.form['password'],
#                 "encrypted_pwd": encryption(request.form['password']),
                "aadhar": request.form['aadhar'],
                "schoolname": request.form['schoolname'],
                "loggedIn": False,
                "coursePointer": "none",
                "subjectPointer": "none",
                "chapterPointer": "none",
                "raise_for_help": "no",
                "question_paper_received":"False",
                "teacherPointer": None,
                "Exam Attending" : None,
                "exam status": "None",
                "dob": request.form['dob'],          # MM-DD-YYYY
                "sex": request.form['sex'],
                "mobile": request.form['mobile'],
                "rollNumber": "none",  # public exam number
                "address": request.form['address'],  # string
                "pincode": request.form['pincode'],  #int
                "class": request.form['class'],
                "section":request.form["section"],
                 "isEnrolled": False,# int
                "testScore": {},
                "examScore": {},
                "enrolledCourses": {},
                "lastAccessed": "none"
            }
        
        db_check_user = collection.find_one(
            {
                "regno":dataAccount["regno"],
                "name":dataAccount["name"]
            }
        ) #check if user account exists in student collection
        
        if not(db_check_user):
            db_response = collection.insert_one(dataAccount)
#             data = {
# #                 "parentId": db_response.inserted_id,
                
#             }
            
#             db_response_account = collection_student.insert_one(data) #insert in students_profile collection
            
            if db_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
        
        else:
            response_string = {"status": "failed","response": "User account already exists"}
        
    return json.dumps(response_string)

# Admin Portal - delete individual student

In [6]:
@app.route('/student/delete',methods=["POST"])
def student_delete():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        
        regno = request.form['regno']
        print(regno)
#         passw = request.form['password']
        
        db_response = collection.find_one({"regno": regno})
        
        print(db_response)
        
        if db_response != None:
            
#             if passw == db_response["password"]:
            
                delete_student = collection.delete_one({"_id": ObjectId(db_response['_id'])})

                return json.dumps({"status": "success", "response": "Student profile deletd for the regno "+regno}) 
            

        else:

            response_string = {"status": "failed","response": "User not found"}
                             
      

# Admin Portal - delete all student based on class and section

In [7]:
@app.route('/student/delete_All',methods=["POST"])
def student_delete_All():
    db = myclient["accounts"]
    
    collection = db["student"]
    
    if request.method == 'POST':
        
        std = request.form["class"]
        
        sec = request.form["section"]
        
        db_response = collection.find({"class":std,"section":sec},{"_id":0})
        
        teachers_doc = []
            
        for doc in db_response:
            teachers_doc.append(doc)
            
        
        if teachers_doc != []:
            
                delete_students = collection.delete_many({"class":std,"section":sec})
   

                response_string = {"status": "success","response":"All students Records are deleted "}
                    
                
                return json.dumps(response_string, indent=4)
          
        else:
                return json.dumps({"status": "failed","response": "User not found"})

# Teacher registration

In [8]:
@app.route('/teacher/registration',methods=["POST"])
def teacher_registration():
#   declaration space
    db = myclient["accounts"]
    collection = db["teacher"]
    data = {}
    response_string = {}
    if request.method == 'POST':
        data = {
            "name": request.form['name'],
            "aadhar": request.form['aadhar'],
            "dob": request.form['dob'],
            "sex": request.form['sex'],
            "regId": request.form['regId'],
            "mobile": request.form['mobile'],
            "password": request.form['password'],  
#             "encrypted_pwd": encryption(request.form['password']),    
            "createdAt": str(datetime.datetime.now()).split(" ")[0], #YYYY-MM-DD
            "enrolledCourses": {},
            "coursePointer": "none",
            "createdTests": {},
            "isExaminer": False,
            "loggedIn": False,
            "pincode": request.form['pincode'] 
        }
        
        db_check_user = collection.find_one({"regId":data["regId"],"dob":data["dob"]}) #check if user account exists
        
        if not(db_check_user):
            db_response = collection.insert_one(data)
            
            if db_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
        
        else:
            response_string = {"status": "failed","response": "User account already exists"}
        
    return json.dumps(response_string)

# Teacher - login

In [9]:
@app.route('/teacher/login',methods=["POST"])
def teacher_login():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        passw = request.form['password']
        db_response = collection.find_one({"regId": regId,"password":passw})
        if db_response != None:
            if db_response['loggedIn'] != True:
                if passw == db_response['password']:
                    update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": False},{"$set":{"loggedIn":True}})
                    if update_result.acknowledged == True and update_result.modified_count == 1:
                        response_string = {"status": "success","response":"ok"}
                    else:
                        response_string = {"status":"failed","response":"database error"}
                    return json.dumps(response_string, indent=4)
                else:
                    return json.dumps({"status": "failed","response":"wrong password"})
            else:
                return json.dumps({"status": "failed","response": "You are logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [10]:
@app.route('/teacher/reset-password',methods=["POST"])
def admin_portal_teacher_reset_password():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        aadhar = request.form['aadhar']
        password = request.form['password']
        db_response = collection.find_one({"regId": regId})
        if db_response != None:
            if aadhar == db_response['aadhar']:
                update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"password":password}})
                if update_result.acknowledged:
                    return json.dumps({"status": "success","response": "ok"})
                return json.dumps({"status": "failed","response": "database error"})
            else:
                return json.dumps({"status": "failed","response": "aadhar didn't match"})
        else:
            return json.dumps({"status": "user not found","response": "User not found"})

# Teacher Logout

In [11]:
@app.route('/teacher/logout',methods=["POST"])
def admin_portal_teacher_logout():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        passw = request.form['password']
        db_response = collection.find_one({"regId": regId,"password":passw})
        if db_response != None:
            if db_response['loggedIn'] != False:
               
                update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": True},{"$set":{"loggedIn":False}})
                if update_result.acknowledged == False and update_result.modified_count == 1:
                    response_string = {"status": "logout success","response":"ok"}

                else:
                    response_string = {"status":"failed","response":"database error"}
                return json.dumps(response_string, indent=4)
               
            else:
                return json.dumps({"status": "failed","response": "You are logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

# Examiner - Registration

In [12]:
@app.route('/examiner/registration',methods=["POST"])
def examiner_registration():
#   declaration space
    db = myclient["accounts"]
    collection = db["examiner"]
    data = {}
    response_string = {}
    if request.method == 'POST':
        data = {
            "name": request.form['name'],
            "hallno":request.form['hallno'],
            "aadhar": request.form['aadhar'],
            "dob": request.form['dob'],
            "sex": request.form['sex'],
            "address": request.form['address'],
            "examiner_id": request.form['examiner_id'],
            "mobile": request.form['mobile'],
            "password": request.form['password'],  
#             "encrypted_pwd": encryption(request.form['password']),    
            "createdAt": str(datetime.datetime.now()).split(" ")[0], #YYYY-MM-DD
            "loggedIn": False,
            "pincode": request.form['pincode'] 
        }
        
        db_check_user = collection.find_one({"examiner_id":data["examiner_id"],"dob":data["dob"]}) #check if user account exists
        
        if not(db_check_user):
            db_response = collection.insert_one(data)
            
            if db_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
        
        else:
            response_string = {"status": "failed","response": "User account already exists"}
        
    return json.dumps(response_string)

# Examiner login

In [13]:
@app.route('/admin-portal/examiner-login',methods=["POST"])
def admin_portal_examiner_login():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        examiner_id = request.form['examiner_id']
        passw = request.form['password']
        db_response = collection.find_one({"examiner_id": examiner_id,"password":passw})
        if db_response != None:
            if db_response['loggedIn'] != True:
                if passw == db_response['password']:
                    update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": False},{"$set":{"loggedIn":True}})
                    if update_result.acknowledged == True and update_result.modified_count == 1:
                        response_string = {"status": "success","response":"ok"}
                        
                    else:
                        response_string = {"status":"failed","response":"database error"}
                    return json.dumps(response_string, indent=4)
                else:
                    return json.dumps({"status": "failed","response":"wrong password"})
                
            elif db_response['loggedIn'] == True:
                response_string = {"status": "successful","response":"ok"}
                return json.dumps(response_string, indent=4)
        else:
            return json.dumps({"status": "failed","response": "User not found"})

# Examiner - reset password

In [14]:
@app.route('/admin-portal/examiner-reset-password',methods=["POST"])
def admin_portal_examiner_reset_password():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        examinerId = request.form['examiner_id']
        aadhar = request.form['aadhar']
        password = request.form['password']
        db_response = collection.find_one({"examiner_id": examinerId})
        if db_response != None:
            if aadhar == db_response['aadhar']:
                update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"password":password}})
                if update_result.acknowledged:
                    return json.dumps({"status": "success","response": "ok"})
                return json.dumps({"status": "failed","response": "database error"})
            else:
                return json.dumps({"status": "failed","response": "aadhar didn't match"})
        else:
            return json.dumps({"status": "user not found","response": "User not found"})

# Examiner - logout

In [15]:
@app.route('/admin-portal/examiner-logout',methods=["POST"])
def admin_portal_examiner_logout():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        examinerId = request.form['examiner_id']
        passw = request.form['password']
        db_response = collection.find_one({"examiner_id": examinerId,"password":passw})
        if db_response != None:
            if db_response['loggedIn'] != False:
               
                update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": True},{"$set":{"loggedIn":False}})
                if update_result.acknowledged == False and update_result.modified_count == 1:
                    response_string = {"status": "logout success","response":"ok"}

                else:
                    response_string = {"status":"failed","response":"database error"}
                return json.dumps(response_string, indent=4)
               
            else:
                return json.dumps({"status": "failed","response": "You are logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

# Admin portal - delete individual teacher

In [16]:
@app.route('/teacher/delete',methods=["POST"])
def teacher_delete():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        
        regId = request.form['regId']
        print(regId)
#         passw = request.form['password']
        
        db_response = collection.find_one({"regId": regId})
        
        print(db_response)
        
        if db_response != None:
            
#             if passw == db_response["password"]:
            
                delete_student = collection.delete_one({"_id": ObjectId(db_response['_id'])})

                return json.dumps({"status": "success", "response": "Teacher profile deletd for the regno "+regId}) 
            

        else:

            response_string = {"status": "failed","response": "User not found"}
                             
      

# Admin portal - delete all teachers

In [17]:
@app.route('/teacher/deleteAll',methods=["POST"])
def teacher_deleteAll():
    db = myclient["accounts"]
    
    collection = db["teacher"]
    
    if request.method == 'POST':
        
        db_response = collection.find({})
        
        teachers_doc = []
            
        for doc in db_response:
            teachers_doc.append(doc)
            
        
        if teachers_doc != []:
            
                delete_teachers = collection.delete_many({})
   

                response_string = {"status": "success","response":"Teacher Records are deleted "}
                    
                
                return json.dumps(response_string, indent=4)
          
        else:
                return json.dumps({"status": "failed","response": "User not found"})

# Admin portal - delete individual examiner

In [18]:
@app.route('/examiner/delete',methods=["POST"])
def examiner_delete():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        
        examinerId = request.form['examiner_id']
        
        db_response = collection.find_one({"examiner_id": examinerId})

        if db_response != None:
                delete_student = collection.delete_one({"_id": ObjectId(db_response['_id'])})

                return json.dumps({"status": "success", "response": "Teacher profile deletd for the regno "+examinerId}) 
            

        else:

            response_string = {"status": "failed","response": "User not found"}
                             
      

# Admin portal - delete all examiners

In [19]:
@app.route('/examiner/deleteAll',methods=["POST"])
def examiner_deleteAll():
    db = myclient["accounts"]
    
    collection = db["examiner"]
    
    if request.method == 'POST':
        
        db_response = collection.find({})
        
        teachers_doc = []
            
        for doc in db_response:
            teachers_doc.append(doc)
            
        
        if teachers_doc != []:
            
                delete_teachers = collection.delete_many({})
   

                response_string = {"status": "success","response":"All examiner records are deleted "}
                    
                
                return json.dumps(response_string, indent=4)
          
        else:
                return json.dumps({"status": "failed","response": "User not found"})

# Student panel

In [20]:
@app.route('/student/getAll',methods=["GET", "POST"])
def student_get():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [21]:
@app.route('/student/admin-approval',methods=["GET", "POST"])
def student_admin_approval():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        db_response = list(collection.find({"isEnrolled":False},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [22]:
@app.route('/student/approved',methods=["GET", "POST"])
def student_admin_approved():
    db = myclient["accounts"]
    collection = db["student"]
    regno = request.form['regno']
    if request.method == 'POST':
        db_response = collection.find({"regno":regno},{"_id":0})
        if db_response != None:
            update_result = collection.update_one({"regno":regno},{"$set":{"isEnrolled":True}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
    return json.dumps({"status": "user not found","response": "User not found"})

# Teacher panel

In [23]:
@app.route('/teacher/getAll',methods=["GET", "POST"])
def teacher_get():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [24]:
@app.route('/teacher/admin-approval',methods=["GET", "POST"])
def teacher_admin_approval():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        db_response = list(collection.find({"isEnrolled":False},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [25]:
@app.route('/teacher/approved',methods=["GET", "POST"])
def teacher_admin_approved():
    db = myclient["accounts"]
    collection = db["teacher"]
    regId = request.form['regId']
    if request.method == 'POST':
        db_response = collection.find({"regId":regId},{"_id":0})
        if db_response != None:
            update_result = collection.update_one({"regId":regId},{"$set":{"isEnrolled":True}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
    return json.dumps({"status": "user not found","response": "User not found"})

# Examiner panel

In [26]:
@app.route('/examiner/getAll',methods=["GET", "POST"])
def examiner_get():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [27]:
@app.route('/examiner/admin-approval',methods=["GET", "POST"])
def examiner_admin_approval():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        db_response = list(collection.find({"isRegistered":False},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [28]:
@app.route('/examiner/approved',methods=["GET", "POST"])
def examiner_admin_approved():
    db = myclient["accounts"]
    collection = db["examiner"]
    examiner_id = request.form['examiner_id']
    if request.method == 'POST':
        db_response = collection.find({"examiner_id":examiner_id},{"_id":0})
        if db_response != None:
            update_result = collection.update_one({"examiner_id":examiner_id},{"$set":{"isRegistered":True}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
    return json.dumps({"status": "user not found","response": "User not found"})

# Subjects retrieval - HSC

In [29]:
@app.route('/student/hsc',methods=["POST"])
def student_hsc():
    db = myclient["Class"]
    collection = db["HSC"]
        
    if request.method == 'POST':
        group = request.form["subjects"]
        db_res = collection.find_one({},{"_id":0})
        
        if db_res != None:
            return json.dumps({group+"":db_res["groups"][group]})
        
    else:
        return json.dumps({"status": "failed","response": "User not found"})

# Subjects retrieval - SSLC

In [30]:
@app.route('/student/sslc',methods=["POST"])
def student_sslc():
    db = myclient["Class"]
    collection = db["SSLC"]
        
    if request.method == 'POST':
        db_res = collection.find_one({},{"_id":0})
        
        if db_res != None:
            return json.dumps({group+"":db_res["subjects"]})
        
    else:
        return json.dumps({"status": "failed","response": "User not found"})

# Confirm courses 

In [31]:
@app.route('/student/confirm-course',methods=["POST"])
def student_confirm_course():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        course = request.json();
        regno = course["regno"]
        std = course["class"]
        section = course["section"]
        subjects = course["subjects"]
        db_response = collection.find_one({"regno":regno,"class":std,"section":section},{"_id":0})
        if db_response != None:
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"enrolledCourses":subjects,"isEnrolled":True}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

# Student course enrollment Web

In [32]:
@app.route('/student/enroll-course',methods=["POST"])
def student_enroll_course():
    db = myclient["Class"]
    collection = db["HSC"]
    sub = {}
    if request.method == 'POST':
        regno = request.form["regno"]
        std = request.form["std"]
        section = request.form["section"]
        group = request.form["group"]
        
        db_res = collection.find_one({},{"_id":0})

        if db_res != None:
            sub = {group+"":db_res["groups"][group]}
            grp = {};
            grp["groups"] = sub
            db = myclient["accounts"]
            collection = db["student"]
            db_response = collection.find_one({"regno":regno,"class":std,"section":section})
            if db_response != None:
                update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"enrolledCourses":grp,"isEnrolled":True}})
                if update_result.acknowledged:
                    return json.dumps({"status": "success","response": "ok"})
                return json.dumps({"status": "failed","response": "database error"})

In [33]:
@app.route('/student/sslc-enroll-course',methods=["POST"])
def student_sslc_enroll_course():
    db = myclient["Class"]
    collection = db["SSLC"]
    sub = {}
    if request.method == 'POST':
        regno = request.form["regno"]
        
        db_res = collection.find_one({},{"_id":0})

        if db_res != None:
            sub = db_res["subjects"]
            db = myclient["accounts"]
            collection = db["student"]
            db_response = collection.find_one({"regno":regno})
            if db_response != None:
                update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"enrolledCourses":sub,"isEnrolled":True}})
                if update_result.acknowledged:
                    return json.dumps({"status": "success","response": "ok"})
                return json.dumps({"status": "failed","response": "database error"})

In [34]:
@app.route('/admin-portal/admin-login',methods=["POST"])
def admin_portal_admin_login():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        adminId = request.form['adminId']
        passw = request.form['password']
        db_response = collection.find_one({"admin_id": adminId,"password":passw})
        if db_response != None:
            if db_response['loggedIn'] != True:
                if passw == db_response['password']:
                    update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": False},{"$set":{"loggedIn":True}})
                    if update_result.acknowledged == True and update_result.modified_count == 1:
                        response_string = {"status": "success","response":"ok"}
                        
                    else:
                        response_string = {"status":"failed","response":"database error"}
                    return json.dumps(response_string, indent=4)
                else:
                    return json.dumps({"status": "failed","response":"wrong password"})
                
            elif db_response['loggedIn'] == True:
                response_string = {"status": "successful","response":"ok"}
                return json.dumps(response_string, indent=4)
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [35]:
@app.route('/admin-portal/admin-registration',methods=["POST"])
def admin_portal_admin_registration():
#   declaration space
    db = myclient["accounts"]
    collection = db["admin"]
    data = {}
    response_string = {}
    if request.method == 'POST':
         data = {
            "name": request.form['name'],
            "aadhar": request.form['aadhar'],
            "dob": request.form['dob'],
            "sex": request.form['sex'],
            "admin_id": request.form['regno'],
            "mobile": request.form['mobile'],
            "password": request.form['password'],
            "schoolname": request.form['schoolname'],
            "address": request.form['address'],
#             "encrypted_pwd": encryption(request.form['password']),    
            "createdAt": str(datetime.datetime.now()).split(" ")[0], #YYYY-MM-DD
            "loggedIn": False,
            "pincode": request.form['pincode'] 
        }
        
        
    db_check_user = collection.find_one({"admin_id":data["admin_id"],"aadhar":data["aadhar"]}) #check if user account exists

    if not(db_check_user):
        db_response = collection.insert_one(data)

        if db_response.acknowledged == True:
            response_string = {"status": "success","response":"ok"}
        else:
            response_string = {"status":"failed","response":"database error. request not acknowledged"}

    else:
        response_string = {"status": "failed","response": "User account already exists"}

    return json.dumps(response_string)

In [36]:
@app.route('/admin-portal/admin-logout',methods=["POST"])
def admin_portal_admin_logout():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        adminId = request.form['adminId']
        passw = request.form['password']
        db_response = collection.find_one({"admin_id": adminId,"password":passw})
        if db_response != None:
            if db_response['loggedIn'] != False:
               
                update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": True},{"$set":{"loggedIn":False}})
                if update_result.acknowledged == False and update_result.modified_count == 1:
                    response_string = {"status": "logout success","response":"ok"}

                else:
                    response_string = {"status":"failed","response":"database error"}
                return json.dumps(response_string, indent=4)
               
            else:
                return json.dumps({"status": "failed","response": "You are logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [37]:
@app.route('/admin-portal/admin-reset-password',methods=["POST"])
def admin_portal_admin_reset_password():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        adminId = request.form['adminId']
        aadhar = request.form['aadhar']
        password = request.form['password']
        db_response = collection.find_one({"admin_id": adminId})
        if db_response != None:
            if aadhar == db_response['aadhar']:
                update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"password":password}})
                if update_result.acknowledged:
                    return json.dumps({"status": "success","response": "ok"})
                return json.dumps({"status": "failed","response": "database error"})
            else:
                return json.dumps({"status": "failed","response": "aadhar didn't match"})
        else:
            return json.dumps({"status": "user not found","response": "User not found"})

In [38]:
@app.route('/admin-portal/login')
def form():
    return render_template('F:/Admin dashboard/index.html')

# Teacher Registration

In [39]:
@app.route('/admin-portal/teacher-registration',methods=["POST"])
def admin_portal_teacher_registration():
#   declaration space
    db = myclient["accounts"]
    collection = db["teacher"]
    data = {}
    response_string = {}
    if request.method == 'POST':
        data = {
            "name": request.form['name'],
            "aadhar": request.form['aadhar'],
            "dob": request.form['dob'],
            "sex": request.form['sex'],
            "regId": request.form['regno'],
            "mobile": request.form['mobile'],
            "password": request.form['password'],
            "schoolname": request.form['schoolname'],
            "address": request.form['address'],
            "pincode": request.form['pincode'],
#             "encrypted_pwd": encryption(request.form['password']),    
            "createdAt": str(datetime.datetime.now()).split(" ")[0], #YYYY-MM-DD
            "enrolledCourses": [],
            "coursePointer": "none",
            "createdTests": {},
            "isExaminer": False,
            "loggedIn": False,
            "pincode": request.form['pincode'] 
        }
        
        db_check_user = collection.find_one({"regId":data["regId"],"dob":data["dob"]}) #check if user account exists
        
        if not(db_check_user):
            db_response = collection.insert_one(data)
            
            if db_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
        
        else:
            response_string = {"status": "failed","response": "User account already exists"}
        
    return json.dumps(response_string)

# Teacher records - deleteAll

In [40]:
@app.route('/admin-portal/teacher/deleteAll',methods=["POST"])
def admin_portal_teacher_deleteAll():
    db = myclient["accounts"]
   
    collection = db["teacher"]
   
    if request.method == 'POST':
       
        db_response = collection.find({})
       
        teachers_doc = []
           
        for doc in db_response:
            teachers_doc.append(doc)
           
       
        if teachers_doc != []:
           
                delete_teachers = collection.delete_many({})
   

                response_string = {"status": "success","response":"Teacher Records are deleted "}
                   
               
                return json.dumps(response_string, indent=4)
         
        else:
                return json.dumps({"status": "failed","response": "User not found"})

# Student - profile

In [41]:
@app.route('/admin-portal/student-profile',methods=["GET", "POST"])
def admin_portal_view_student_profile():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']
        db_response = (collection.find_one({"regno":regno},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [42]:
@app.route('/admin-portal/edit-student-profile',methods=["GET", "POST"])
def admin_portal_student_edit_profile():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']
        db_response = (collection.find_one({"regno":regno},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [43]:
@app.route('/admin-portal/update-student-profile',methods=["GET", "POST"])
def admin_portal_student_update_profile():
    db = myclient["accounts"]
    collection = db["student"]
    admin_name = ''
    adminid = ''
    if request.method == 'POST':
        adminid = request.form['regno']
        
        admin_name = request.form['name']
        admin_password = request.form['password']
        admin_class =  request.form['class']
        admin_section = request.form['section']
        admin_aadhar =  request.form['aadhar']
        admin_mobile = request.form['mobile']
        admin_schoolname = request.form['schoolname']
        admin_dob =  request.form['dob']
        admin_gender = request.form['gender']
        admin_address = request.form['address']
        admin_pincode = request.form['pincode']
        type(admin_name)
        db_response = (collection.find_one({"regno":adminid}))
        if db_response != None:
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"class":admin_class,"section":admin_section,"password":admin_password,"name":admin_name,"aadhar":admin_aadhar,"mobile":admin_mobile,"dob":admin_dob,"sex":admin_gender,"schoolname":admin_schoolname,"address":admin_address,"pincode":admin_pincode}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
    return json.dumps({"status": "user not found","response": "User not found"})

# Teacher - profile

In [44]:
@app.route('/admin-portal/teacher-profile',methods=["GET", "POST"])
def admin_portal_teacher_view_profile():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        db_response = (collection.find_one({"regId":regId},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [45]:
@app.route('/admin-portal/edit-teacher-profile',methods=["GET", "POST"])
def admin_portal_teacher_edit_profile():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        db_response = (collection.find_one({"regId":regId},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [46]:
@app.route('/admin-portal/update-teacher-profile',methods=["GET", "POST"])
def admin_portal_teacher_update_profile():
    db = myclient["accounts"]
    collection = db["teacher"]
    admin_password = "";
    if request.method == 'POST':
        adminid = request.form['regId']
        admin_name = request.form['name']
        admin_password = request.form['password']
        admin_aadhar =  request.form['aadhar']
        admin_mobile = request.form['mobile']
        admin_schoolname = request.form['schoolname']
        admin_dob =  request.form['dob']
        admin_gender = request.form['gender']
        admin_address = request.form['address']
        admin_pincode = request.form['pincode']
        db_response = (collection.find_one({"regId":adminid}))
        if db_response != None:
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"password":admin_password,"name":admin_name,"aadhar":admin_aadhar,"mobile":admin_mobile,"dob":admin_dob,"sex":admin_gender,"schoolname":admin_schoolname,"address":admin_address,"pincode":admin_pincode}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
    return json.dumps({"status": "user not found","response": "User not found"})

# Examiner - profile

In [47]:
@app.route('/admin-portal/examiner-profile',methods=["GET", "POST"])
def admin_portal_examiner_view_profile():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        examinerId = request.form['examiner_id']
        db_response = (collection.find_one({"examiner_id":examinerId},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [48]:
@app.route('/admin-portal/edit-examiner-profile',methods=["GET", "POST"])
def admin_portal_examiner_edit_profile():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        examinerId = request.form['examiner_id']
        db_response = (collection.find_one({"examiner_id":examinerId},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [49]:
@app.route('/admin-portal/update-examiner-profile',methods=["GET", "POST"])
def admin_portal_examiner_update_profile():
    db = myclient["accounts"]
    collection = db["examiner"]
    admin_password = "";
    if request.method == 'POST':
        examinerid = request.form['examiner_id']
        examiner_name = request.form['name']
        examiner_hallno = request.form['hallno']
        examiner_password = request.form['password']
        examiner_aadhar =  request.form['aadhar']
        examiner_mobile = request.form['mobile']
        examiner_schoolname = request.form['schoolname']
        examiner_dob =  request.form['dob']
        examiner_gender = request.form['gender']
        examiner_address = request.form['address']
        examiner_pincode = request.form['pincode']
        db_response = (collection.find_one({"examiner_id":examinerid}))
        if db_response != None:
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"password":examiner_password,"hallno":examiner_hallno,"name":examiner_name,"aadhar":examiner_aadhar,"mobile":examiner_mobile,"dob":examiner_dob,"sex":examiner_gender,"schoolname":examiner_schoolname,"address":examiner_address,"pincode":examiner_pincode}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
    return json.dumps({"status": "user not found","response": "User not found"})

# Teacher enrollment

In [50]:
@app.route('/admin-portal/teacher-course-enrollment',methods=["POST"])
def admin_portal_teacher_course_enrollment():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        courses = request.json["subjects"]
        regId = request.json["regId"]
        
#         regId = json.load(request.form['courses'])["regId"]
        db_response = collection.find_one({"regId": regId})
        print(regId)
        if db_response != None:
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"enrolledCourses":courses}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
        else:
            return json.dumps({"status": "user not found","response": "User not found"})

# Admin - profile

In [51]:
@app.route('/admin-portal/admin-profile',methods=["GET", "POST"])
def admin_portal_admin_profile():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        adminId = request.form['adminId']
        db_response = (collection.find_one({"admin_id":adminId},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

# Admin - Edit profile

In [52]:
@app.route('/admin-portal/admin-edit-profile',methods=["GET", "POST"])
def admin_portal_admin_edit_profile():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        adminId = request.form['adminId']
        db_response = (collection.find_one({"admin_id":adminId},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

# Admin - Update Profile

In [53]:
@app.route('/admin-portal/admin-update-profile',methods=["GET", "POST"])
def admin_portal_admin_update_profile():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        adminid = request.form['admin_id']
        admin_name = request.form['name']
        admin_password = request.form['password']
        admin_aadhar =  request.form['aadhar']
        admin_mobile = request.form['mobile']
        admin_schoolname = request.form['schoolname']
        admin_dob =  request.form['dob']
        admin_gender = request.form['gender']  
        admin_address = request.form['address']        
        admin_pincode = request.form['pincode']
        db_response = (collection.find_one({"admin_id":adminid}))
        if db_response != None:
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"password":admin_password,"name":admin_name,"aadhar":admin_aadhar,"mobile":admin_mobile,"dob":admin_dob,"sex":admin_gender,"schoolname":admin_schoolname,"address":admin_address,"pincode":admin_pincode}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
    return json.dumps({"status": "user not found","response": "User not found"})

# Update content details

In [54]:
@app.route('/update/fileDetails',methods=["POST"])
def update_fileDetails():
#   declaration space
    db = myclient["Class"]
    collection = db["contents"]
    data = {
        "regId" : request.form["regId"],
        "createdAt" : request.form["created_at"],
        "filename" : request.form["file_name"],
        "class" : request.form["class"],
        "section" : request.form["section"],
        "schoolname" : request.form["schoolname"]
    }
    
    print(request.form["schoolname"])
    response_string = {}
    if request.method == 'POST':
        db_response = collection.insert_one(data)

        if db_response.acknowledged == True:
            response_string = {"status": "success","response":"ok"}
        else:
            response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return json.dumps(response_string)

# Get All uploaded file information

In [55]:
@app.route('/filesData/getAll',methods=["GET", "POST"])
def files_get():
    db = myclient["Class"]
    collection = db["contents"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

# Post Assignment

In [56]:
@app.route('/teacher/post-assignments',methods=["POST"])
def insert_assignments():
#   declaration space
    db = myclient["Class"]
    collection = db["assignments"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        insert_response = collection.insert_one(incoming_data)
        if insert_response.acknowledged == True:
            response_string = {"status": "success","response":"ok"}
        else:
            response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [57]:
@app.route('/teacher/sslc-course-enrollment',methods=["POST"])
def teacher_sslc_course_enrollment():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        sub = json.loads(request.form['name'])
        regId = sub['regId']
        std = sub['std']
        sec = sub['sec']
        subjects = sub['subjects']
        db_response = collection.find_one({"regId":regId})
        courses = db_response['enrolledCourses']
        
        
        if db_response != None:
            courses[std][sec] = subjects
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"enrolledCourses":courses}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
        else:
            response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

# Mark Attendance

In [58]:
@app.route('/student/mark-attendance',methods=["POST"])
def student_attendance():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']
        isAttended = request.form['isAttended']
        db_response = collection.find_one({"regno": regno})
        if db_response != None:
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"isAttended":isAttended}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
        else:
            return json.dumps({"status": "user not found","response": "User not found"})

# File uploading

In [59]:
@app.route('/assignment/uploads', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        f = request.files['file']
        filename = f.filename
        
        if filename != '':
            file_ext = os.path.splitext(filename)[1]
            if file_ext not in app.config['UPLOAD_EXTENSIONS']:
                return json.dumps({"status": "failed","response": "File Type not accepted"})
        else:
            return json.dumps({"status": "failed","response": "No file contents"})
        
        f.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        
        return json.dumps({"status": "success","response": "uploaded successfully", "data": parse_text(UPLOAD_FOLDER+'/'+filename)})
    
    return json.dumps({"status": "failed","response": "Permission Denied"})

In [60]:
import tika
import re
from tika import parser
from pdfrw import PdfReader
import pymongo
import operator
import json

def parse_text(filename):
    parsed_pdf = parser.from_file(filename)
    text = parsed_pdf['content']
    
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = url_pattern.sub(r'', text)
    
    text = text.strip() 
    
    #to remove unnecessary breaklines
    text = re.sub(r'(?<!\n)\n(?![\n\t])', ' ', text.replace('\r', ''))
    print(text)
    return text

# Examination - Time Table

In [61]:
@app.route('/exam/time-table',methods=["POST"])
def insert_timetable():
    db = myclient["Class"]
    collection = db["exam - time table"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user'],"schoolname": incoming_data['schoolname'],"exam-name": incoming_data['exam-name'],"class": incoming_data['class']},{"_id": 0})
       
        if existing_data != None:
            if(incoming_data["schoolname"] == existing_data["schoolname"] and incoming_data["exam-name"] == existing_data["exam-name"] and incoming_data["class"] == existing_data["class"]):
                collection.delete_many(existing_data)
                collection.insert_one(incoming_data)
                response_string = {"status": "successful","response":"updated"}
            else:
                insert_response = collection.insert_one(incoming_data)
                if insert_response.acknowledged == True:
                    response_string = {"status": "success","response":"ok"}
                else:
                    response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"inserted"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
           
    return make_response(jsonify(response_string),200)


# Get Exam Time Table

In [62]:
@app.route('/exam-time-table/getAll',methods=["GET", "POST"])
def exam_time_table_getAll():
    db = myclient["Class"]
    collection = db["exam - time table"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "Time table not found","response": "Time table not found"})

# Teacher - class time table

In [63]:
@app.route('/teacher/class-time-table',methods=["POST"])
def teacher_class_time_table():
    db = myclient["Class"]
    collection = db["teacher - time table"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user'],"schoolname": incoming_data['schoolname']},{"_id": 0})
       
        if existing_data != None:
            if(incoming_data["schoolname"] == existing_data["schoolname"]):
                collection.delete_many(existing_data)
                collection.insert_one(incoming_data)
                response_string = {"status": "successful","response":"updated"}
            else:
                insert_response = collection.insert_one(incoming_data)
                if insert_response.acknowledged == True:
                    response_string = {"status": "success","response":"ok"}
                else:
                    response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"inserted"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
           
    return make_response(jsonify(response_string),200)


# Get Teacher time tables

In [64]:
@app.route('/teacher-time-table/getAll',methods=["GET", "POST"])
def teacher_time_table_getAll():
    db = myclient["Class"]
    collection = db["teacher - time table"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "Time table not found","response": "Time table not found"})

# Student - class time table

In [65]:
@app.route('/student/class-time-table',methods=["POST"])
def student_class_time_table():
    db = myclient["Class"]
    collection = db["student - time table"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user'],"schoolname": incoming_data['schoolname'],"class": incoming_data['class'],"section": incoming_data['section']},{"_id": 0})
       
        if existing_data != None:
            if(incoming_data["schoolname"] == existing_data["schoolname"] and incoming_data["class"] == existing_data['class'] and incoming_data["section"] == existing_data['section']):
                collection.delete_many(existing_data)
                collection.insert_one(incoming_data)
                response_string = {"status": "successful","response":"updated"}
            else:
                insert_response = collection.insert_one(incoming_data)
                if insert_response.acknowledged == True:
                    response_string = {"status": "success","response":"ok"}
                else:
                    response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"inserted"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
           
    return make_response(jsonify(response_string),200)


# Get All Student Time Table

In [66]:
@app.route('/student-time-table/getAll',methods=["GET", "POST"])
def student_time_table_getAll():
    db = myclient["Class"]
    collection = db["student - time table"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "Time table not found","response": "Time table not found"})

# Notifications

In [67]:
@app.route('/notifications',methods=["POST"])
def notifications():
    db = myclient["Class"]
    collection = db["notifications"]
    if request.method == 'POST':
        posted_time = request.form['posted-time']
        posted_date = request.form['posted-date']
        schoolname = request.form['schoolname']
        exam_name = request.form['exam-name']
        admin_id = request.form['admin-id']
        notification = request.form['notification']
        insert_result = collection.insert_one({"admin-id": admin_id,"schoolname": schoolname,"exam-name": exam_name,"notification":notification,"posted-date": posted_date,"posted-time":posted_time},{"_id": 0})
        if insert_result.acknowledged == True:
            return json.dumps({"status": "successfully notified","response": "ok"})
        return json.dumps({"status": "failed","response": "database error"})

# Get All Notifications

In [68]:
@app.route('/notifications/getAll',methods=["GET", "POST"])
def notifications_getAll():
    db = myclient["Class"]
    collection = db["notifications"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "No notifications","response": "No notifications found"})

# Starting the server

In [ ]:
if __name__ == '__main__':
    app.run("127.0.0.1",5487)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5487/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Aug/2021 11:01:46] "POST /admin-portal/admin-registration HTTP/1.1" 400 -


# Mongo Files

In [ ]:
fs = GridFS(MongoClient().testgrid)